# Pi3 mini-uart

**Configure mini-uart (miniserial) for gpio use (disable console).**

* [Official Documentation](https://www.raspberrypi.org/documentation/configuration/uart.md)
* [Detailed Instructions](https://www.circuits.dk/setup-raspberry-pi-3-gpio-uart/)
* Default Pi3 configuration:
   * primary serial for BT @ /dev/ttyAMA0
   * miniserial for console @ /dev/ttyS0, availabe on GPIO pins 8 & 10
* mini UART features:
   * 8 symbols deep FIFOs for receive and transmit.
   * 7 or 8 bit operation.
   * 1 start and 1 stop bit.
   * No parities.
   * Break generation.
   * SW controlled RTS, SW readable CTS.
   * Auto flow control with programmable FIFO level.
   * Baudrate derived from system clock.

## Enable mini UART for GPIO

1. Enable: add `enable_uart=1` to `boot/config.txt`
   * `sudo nano /boot/config.txt`
2. Reboot & look for `/dev/serial0 -> ttyS0`
   * `/dev/serial1 -> /dev/ttyAMA0` is used for BT

## Disable the Console

For Raspberry Pi 3’s the command is referencing /dev/ttyS0:

In [1]:
!sudo systemctl stop serial-getty@ttyS0.service
!sudo systemctl disable serial-getty@ttyS0.service

In [4]:
# remove 'console=serial0,115200' from `/boot/cmdline.txt`

%cat  /boot/cmdline.txt

console=tty1 root=PARTUUID=4c969853-02 rootfstype=ext4 elevator=deadline fsck.repair=yes rootwait


In [ ]:
!sudo reboot now

## Tests

Short RX to TX on header: https://pinout.xyz/

In [9]:
import serial, time

baudrate = 9600
baudrate = 115200
baudrate = 4000_000

with serial.Serial("/dev/serial0", baudrate=baudrate, timeout=3.0) as dev:
    for i in range(10):
        dev.write(f"{i} Hello world!".encode())
        time.sleep(0.1)
        while dev.in_waiting:
            rcv = dev.read(dev.in_waiting)
            print(rcv.decode())

0 Hello world!
1 Hello world!
2 Hello world!
3 Hello world!
4 Hello world!
5 Hello world!
6 Hello world!
7 Hello world!
8 Hello world!
9 Hello world!


In [23]:
import serial, os, time

N = 4000

baudrate = 9600
baudrate = 115200
baudrate = 1000_000

with serial.Serial("/dev/serial0", baudrate=baudrate, timeout=3.0) as dev:
    for i in range(2):
        data = os.urandom(N)
        dev.write(data)
        while dev.in_waiting < N:
            time.sleep(0.001)
        recv = dev.read(N)
        if recv != data:
            print(f"mismatch: {data} != {recv}")
        
print("MATCH!")

MATCH!


## Flow Control

https://ethertubes.com/raspberry-pi-rts-cts-flow-control/